In [2]:
import os
from sqlalchemy import insert, select

import settings
from camtrappy.db.schema import Video, Location, Project
from camtrappy.db.testdb import Session
from camtrappy.io import video, parsing

DATA_DIR = settings.data_dir_real

In [3]:
project = parsing.Project(projectfolder=None, datafolder=DATA_DIR)
project._data.items()

dict_items([('1_Tauberbischofsheim', ['20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_010534_BEE9.mkv', '20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_011034_B0E3.mkv', '20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_012036_B475.mkv', '20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_011535_FDCB.mkv', '20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_010033_1492.mkv'])])

In [2]:
# # combine sessionmaker Session, with-statement & begin() to start session, handle exceptions, rollback, commit and close at once
# with Session.begin() as session:
#     session.add(object)

Fill db with videos

In [4]:
# videos = video.video_paths(data, ['mkv'])

# new_videos = [Video(path=video) for video in videos]
# with Session.begin() as session:
#     session.add_all(new_videos)

In [2]:
project = Project(name='Testproject')
videos = video.video_paths(data, ['mkv'])

for video_path in videos:
    location_name = video.location_from_path(video_path)
    location = Location(name=location_name, project=project)
    vid = Video(path=video_path, location=location)
    
    

In [7]:
videos = video.video_paths(data, ['mkv'])
videos

['1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_010534_BEE9.mkv',
 '1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_011034_B0E3.mkv',
 '1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_012036_B475.mkv',
 '1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_011535_FDCB.mkv',
 '1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_010033_1492.mkv']

In [11]:
with Session.begin() as session:
    project = Project(name='Testproject')
    session.add(project)

videos = video.video_paths(data, ['mkv'])
locations = [video.location_from_path(vid) for vid in videos]
locations_videos = {location: video for location, video in zip(locations, videos)}
print(locations_videos)

# project = Project(name='Testproject')
# new_locations = [Location(name=location, project=project) for location in locations]
# new_videos = [Video(video=path, location=) for path in videos]

# with Session.begin() as session:
    
#     session.add(project)
    # result = session.query(Project).filter(Project.name == 'Testproject').first()
    # print(result.id, result.name)
    

2021-07-16 20:22:09,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-16 20:22:09,630 INFO sqlalchemy.engine.Engine INSERT INTO projects (name, date_finished) VALUES (?, ?)
2021-07-16 20:22:09,630 INFO sqlalchemy.engine.Engine [cached since 4287s ago] ('Testproject', None)
2021-07-16 20:22:10,091 INFO sqlalchemy.engine.Engine COMMIT
{'1_Tauberbischofsheim': '1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_010033_1492.mkv'}


In [4]:
videos = video.video_paths(data, ['mkv'])
videos_by_location = video.videos_by_location(0, videos)
# locations = {video.location_from_path(vid) for vid in videos}


project = Project(name='Testproject')
new_locations = [Location(name=location, project=project) for location in locations]

AttributeError: 'NoneType' object has no attribute 'append'

In [5]:
x = new_locations[0]
x.name, x.project.name

DetachedInstanceError: Instance <Location at 0x8ce4d90> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: http://sqlalche.me/e/14/bhk3)